In [ ]:
import copy
import json
import math
import warnings
from bisect import bisect_left
from collections import Counter

import numpy as np
import pandas as pd
import pandas.io.sql as psql
import psycopg2 as pg
from sqlalchemy import create_engine

warnings.filterwarnings('ignore')

In [290]:
engine = create_engine('postgresql://user@localhost:5432/mydb')
connection = pg.connect("host=188.72.109.162 dbname=optimus_post_db user=reader password=aboba")
cursor = connection.cursor()

In [291]:
cursor.execute("""
    SELECT table_name FROM information_schema.tables
    WHERE table_schema = 'public'
""")
for table in cursor.fetchall():
    print(table)

('schema_migrations',)
('ar_internal_metadata',)
('active_storage_blobs',)
('active_storage_attachments',)
('mos_kiosks',)
('mos_shopping_facilities',)
('mos_multifunctional_centers',)
('mos_libraries',)
('mos_culture_houses',)
('mos_datasets',)
('vacuum_cleaner_statuses',)
('houses',)
('mos_rows',)
('mos_sports_facilities',)
('areas',)
('districts',)
('active_storage_variant_records',)
('automatic_post_offices',)
('placement_object_types',)


In [ ]:
kioskDf = psql.read_sql('SELECT * FROM mos_kiosks', connection)

In [ ]:
shoppingDf = psql.read_sql('SELECT * FROM mos_shopping_facilities', connection)

In [ ]:
mfcDf = psql.read_sql('SELECT * FROM mos_multifunctional_centers', connection)

In [ ]:
librariesDf = psql.read_sql('SELECT * FROM mos_libraries', connection)

In [ ]:
cultureDf = psql.read_sql('SELECT * FROM mos_culture_houses', connection)

In [ ]:
sportsDf = psql.read_sql('SELECT * FROM mos_sports_facilities', connection)

In [ ]:
print(kioskDf.shape)
print(shoppingDf.shape)
print(mfcDf.shape)
print(librariesDf.shape)
print(cultureDf.shape)
print(sportsDf.shape)

(1815, 21)
(2515, 22)
(139, 21)
(270, 20)
(78, 17)
(346, 17)


In [ ]:
datasetsDf = psql.read_sql('SELECT * FROM mos_datasets', connection)

In [ ]:
housesDf = psql.read_sql('SELECT * FROM houses', connection)

In [ ]:
from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import PolynomialFeatures

print(len(housesDf))
df_houses = housesDf.fillna(0).groupby('address').agg({'id': min, 'square': max, 'year': max, 'floors': max, 'entrances': max, 'rooms': max}).reset_index().sort_values('id').set_index('id')
print(len(df_houses))
# df_houses.isna().sum()
# print(len(df_houses), len(df_houses[df_houses['square'] < 1]), len(df_houses[df_houses['floors'] < 1]), len(df_houses[df_houses['entrances'] < 1]), len(df_houses[df_houses['rooms'] < 1]))
# df_houses[((df_houses['square'] > 0) & (df_houses['floors'] > 0) & (df_houses['year'] > 0)) & (df_houses['rooms'] > 0)]

model = LinearRegression()
# poly = PolynomialFeatures(degree=2, include_bias=True)

# stage 1
X = df_houses[((df_houses['square'] > 0) & (df_houses['floors'] > 0)) & (df_houses['rooms'] > 0)]
model.fit(X[['square', 'floors']], X['rooms'])
X = df_houses[((df_houses['square'] > 0) & (df_houses['floors'] > 0)) & (df_houses['rooms'] < 1)]
idx1 = X.index
preds1 = model.predict(X[['square', 'floors']])

# stage 2
X = df_houses[((df_houses['square'] > 0) & (df_houses['floors'] < 1)) & (df_houses['rooms'] > 0)]
model.fit(X[['square']], X['rooms'])
X = df_houses[((df_houses['square'] > 0) & (df_houses['floors'] < 1)) & (df_houses['rooms'] < 1)]
idx2 = X.index
preds2 = model.predict(X[['square']])

# stage 3
X = df_houses[((df_houses['square'] < 0.1) & (df_houses['floors'] > 0)) & (df_houses['rooms'] > 0)]
model.fit(X[['floors']], X['rooms'])
X = df_houses[((df_houses['square'] < 0.1) & (df_houses['floors'] > 0)) & (df_houses['rooms'] < 1)]
idx3 = X.index
preds3 = model.predict(X[['floors']])

# setting results
df_houses.loc[idx1, 'rooms'] = list(map(round, preds1))
df_houses.loc[idx2, 'rooms'] = list(map(round, preds2))
df_houses.loc[idx3, 'rooms'] = list(map(round, preds3))
# df_houses = df_houses[df_houses['rooms'] > 0]
df_houses['rooms'].replace(0, df_houses['rooms'].median(), inplace=True)
assert len(df_houses[df_houses['rooms'] < 1]) == 0
df_houses.to_csv('houses_new.csv')

33398
32153


In [ ]:
banned_words = [
    'справочник',
    'рейтинг',
    'сведения',
    'работы',
    'график',
    'календарь',
    'маршрут',
    'реестр',
    'перечень'
]
interesting_ids = [493, 495, 498, 499, 501, 502, 503, 504, 505, 1798, 506, 509, 512, 515, 1799, 517, 518, 526, 527, 528, 529, 530, 531, 536, 537, 538, 539, 540, 541, 542, 543, 544, 546, 547, 548, 551, 552, 553, 554, 555, 556, 557, 559, 563, 564, 565, 566, 567, 568, 569, 570, 572, 573, 574, 575, 576, 577, 582, 585, 586, 587, 588, 611, 621, 622, 624, 629, 1800, 673, 745, 753, 754, 1047, 842, 1831, 898, 905, 908, 910, 1832, 918, 1045, 1833, 1095, 1834, 1181, 1193, 1255, 1256, 1260, 1344, 1464, 1661, 1681, 1682, 1775, 1765, 1766, 1767, 1768, 1770, 1793, 1772, 1773, 1774, 1794, 1776, 1777, 1778, 1779, 1796, 1797, 1780, 1781, 1783, 1784, 1787, 1788, 1789, 1790, 1791, 1792, 1801, 1802, 1803, 1804, 1805, 1806, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1826, 1827, 1828, 1829, 1830, 1835, 1836, 1837, 1838, 1840, 1842, 1843, 1844, 1845, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1881, 1903, 1904, 2985, 2115, 2116, 2249, 2254, 2263, 2346, 2265, 2266, 2267, 2268, 2269, 2343, 2351, 2357, 2563, 2601, 2624, 2704, 3015, 3304, 3326, 60622, 60624, 60762, 60801, 61861, 62201, 62207, 62890, 62503, 62841, 62888]
interesting_ids == set(interesting_ids)
datasets_filtered = []
for index, row in datasetsDf.iterrows():
    if row['contains_geodata'] == True:
        caption = row['caption'].lower()
        for w in banned_words:
            if w in caption:
                continue
        datasets_filtered.append(row)
datasets_filtered = [row for row in datasets_filtered if row['id'] in interesting_ids]
datasets_filtered.sort(key=lambda x: x["id"])
len(datasets_filtered)

211

In [ ]:
print(*[f"{row['id']}: {row['caption']}" for row in datasets_filtered], sep='\n')

In [ ]:
ds_duplicates = psql.read_sql("SELECT global_id, array_to_string(array_agg(mos_dataset_id), ', ') AS datasets FROM mos_rows GROUP BY global_id HAVING global_id IS NOT NULL AND COUNT(*) > 1", connection)

In [344]:
duplicates = set(map(lambda s: tuple(sorted(map(int, s.split(', ')))), ds_duplicates['datasets'].tolist()))
duplicates = [x for x in duplicates if any(id in interesting_ids for id in x)]
# len(duplicates)
# sorted(duplicates)
datasets_with_duplicates = sorted(set([item for sublist in duplicates for item in sublist]))
print(len(datasets_with_duplicates), datasets_with_duplicates)

123 [538, 539, 540, 541, 542, 543, 551, 552, 553, 561, 562, 564, 565, 566, 567, 568, 569, 572, 573, 574, 575, 576, 577, 579, 582, 585, 587, 596, 1765, 1766, 1767, 1768, 1769, 1770, 1771, 1772, 1773, 1774, 1775, 1776, 1777, 1778, 1779, 1780, 1781, 1782, 1783, 1784, 1785, 1788, 1790, 1792, 1793, 1794, 1796, 1797, 1798, 1799, 1800, 1801, 1802, 1803, 1804, 1805, 1806, 1808, 1809, 1810, 1811, 1812, 1813, 1814, 1815, 1816, 1817, 1818, 1819, 1820, 1821, 1822, 1823, 1824, 1826, 1827, 1828, 1830, 1831, 1832, 1833, 1834, 1835, 1836, 1837, 1838, 1840, 1842, 1843, 1844, 1845, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1856, 1857, 1858, 1859, 1860, 1861, 1862, 1903, 1904, 2270, 2704, 3304, 60622, 61360, 62361]


In [ ]:
with open('analytics.txt', 'r') as fin:
    datasets_selected = fin.read().split('__skip__')[1].strip().split('\n\n')
dataset_ids_by_group = {
    group_name: list(map(lambda x: int(x.split(': ')[0]), lst.split('\n')))
    for group_name, lst in map(lambda x: x.split(':\n', 1), datasets_selected)
}

connection = pg.connect("host=188.72.109.162 dbname=optimus_post_db user=reader password=aboba")
def extract_points(ids):
    def get_polygon_center(coords):
        x, y = zip(*coords[0][:-1])
        return sum(x) / len(x), sum(y) / len(y)
    
    def get_coords(geo):
        match geo['type']:
            case 'Point':
                return [tuple(geo['coordinates'])]
            case 'Polygon':
                return [get_polygon_center(geo['coordinates'])]
            case 'MultiPoint':
                return [tuple(coords) for coords in geo['coordinates']]
            case 'MultiPolygon':
                return [get_polygon_center(coords) for coords in geo['coordinates']]
            case _:
                raise ValueError(geo['type'])

    def to_float(x):
        return float(x.replace(',', '.')) if isinstance(x, str) else x
    
    geo_data = psql.read_sql(f'SELECT geo_data, data FROM mos_rows WHERE mos_dataset_id IN {ids}', connection)
    # todo: better remove duplicates by global_id
    geo_data_set = set()
    for geo, data in zip(geo_data['geo_data'], geo_data['data']):
        if geo:
            geo_data_set.update(get_coords(geo))
        elif 'X' in data:
            geo_data_set.add((to_float(data['X']), to_float(data['Y'])))
        elif 'X_WGS84' in data:
            geo_data_set.add((to_float(data['X_WGS84']), to_float(data['Y_WGS84'])))
        elif 'Longitude_WGS84' in data:
            geo_data_set.add((to_float(data['Longitude_WGS84']), to_float(data['Latitude_WGS84'])))
        else:
            print(data.keys())
    return list(geo_data_set)

coords_by_group = {
    group_name: extract_points(tuple(ids))
    for group_name, ids in dataset_ids_by_group.items()
}

In [549]:
with open('coordinates_by_group.json', 'w') as fout:
    json.dump(coords_by_group, fout)
[(k, len(v)) for k, v in coords_by_group.items()]

[('activities', 8867),
 ('entertainment', 10343),
 ('shops', 10899),
 ('services', 13701),
 ('health', 1125),
 ('kindergartens', 2848),
 ('education', 7687),
 ('food', 24282),
 ('transport_public', 7971),
 ('transport_other', 4099),
 ('work_and_housing', 1750),
 ('religion', 578)]

In [588]:
addr_to_coords = pd.read_csv('addresses_to_coordinates.csv').set_index('address').to_dict()['coordinates']
df_houses['coordinates'] = df_houses['address'].apply(lambda x: tuple(map(float, addr_to_coords[x][1:-1].split(', '))) if len(addr_to_coords[x]) > 2 else pd.NA)
df_houses.dropna(inplace=True)
df_houses

,address,square,year,floors,entrances,rooms,coordinates
id,,,,,,,
1,"пр-кт. Мира, д. 188 Б, к. 1, Москва",54804.10,2014,58,1.0,370.0,"(37.65806954672649, 55.83441355)"
2,"пр-кт. Мира, д. 188Б, к. 3, Москва",53253.20,2015,58,1.0,356.0,"(37.6584768, 55.8355402)"
3,"пер. Чапаевский, д. 3, Москва",256740.60,2006,57,14.0,992.0,"(37.52038218638005, 55.79818205)"
4,"ул. Мосфильмовская, д. 8, Москва",137500.80,2011,53,2.0,564.0,"(37.527035300180884, 55.72377795)"
5,"ул. Мосфильмовская, д. 70, Москва",341272.69,2006,50,7.0,1046.0,"(37.5088701, 55.7168266)"
...,...,...,...,...,...,...,...
32151,"пер. Переведеновский, д. 4, стр. 1, Москва",1512.60,1916,0,0.0,46.0,"(37.690297, 55.7802424)"
32152,"ул. Руставели, д. 6, к. 5, Москва",2505.00,1966,0,0.0,58.0,"(37.588301742592506, 55.8121977)"
32153,"пер. Староконюшенный, д. 10/10, стр. 1, Москва",3066.00,0,0,0.0,64.0,"(37.59626305140152, 55.7451523)"


In [ ]:
connection = pg.connect("host=188.72.109.162 dbname=optimus_post_db user=reader password=aboba")
point_type = 'mos_kiosks'  # mos_kiosks, mos_shopping_facilities, mos_multifunctional_centers, mos_libraries, mos_culture_houses, mos_sports_facilities
post_points = psql.read_sql(f'SELECT * FROM {point_type}', connection).set_index('id')

In [591]:
class FeatureGeneratorCountHousesInRadius:
    def __init__(self, houses, radiuses_in_km=[0.1, 0.2, 0.4, 0.8]):
        self.houses = houses
        self.radiuses = radiuses_in_km
        self.R = 6373.0  # approximate radius of earth in km

    def calc_dist_in_km(self, coord1, coord2):
        lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
        lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
        a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return self.R * c

    def __call__(self, target_coords):
        """
        returns generator of pairs: (feature_name, feature_values)
        """
        house_rooms = self.houses['rooms']
        house_coordinates = self.houses['coordinates']
        distances = [
            [sorted([(self.calc_dist_in_km(trg_coord, hou_coord), n_rooms) for hou_coord, n_rooms in zip(house_coordinates, house_rooms)], key=lambda x: x[0])]
            for trg_coord in target_coords
        ]
        ds, rooms = zip(*distances)
        for radius in self.radiuses:
            idx = bisect_left(ds, radius)
            yield f'house_rooms_in_{radius}_km', sum(rooms[:idx])


class FeatureGeneratorCountObjectsInRadius:
    def __init__(self, data, radiuses_in_km=[0.1, 0.2, 0.4, 0.8]):
        """
        data: {'object_group': [(lat1, long1), (lat2, long2), ...]}
        """
        self.data = data
        self.radiuses = radiuses_in_km
        self.R = 6373.0  # approximate radius of earth in km

    def calc_dist_in_km(self, coord1, coord2):
        lat1, lon1 = math.radians(coord1[0]), math.radians(coord1[1])
        lat2, lon2 = math.radians(coord2[0]), math.radians(coord2[1])
        a = math.sin((lat2 - lat1) / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin((lon2 - lon1) / 2) ** 2
        c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return self.R * c

    def __call__(self, target_coords):
        """
        returns generator of pairs: (feature_name, feature_values)
        """
        for group_name, object_coords in self.data.items():
            distances = [
                sorted(self.calc_dist_in_km(trg_coord, obj_coord) for obj_coord in object_coords)
                for trg_coord in target_coords
            ]
            for radius in self.radiuses:
                yield f'{group_name}_in_{radius}_km', [bisect_left(ds, radius) for ds in distances]

In [592]:
def generate_features(df_points):
    feature_generators = [
        FeatureGeneratorCountHousesInRadius(df_houses),
        FeatureGeneratorCountObjectsInRadius(coords_by_group),
        # todo: add traffic features
    ]
    for feat_gen in feature_generators:
        for feat_name, feat_values in feat_gen(df_points['coordinates']):
            df_points[feat_name] = feat_values

post_points['coordinates'] = [tuple(geo['coordinates']) for geo in post_points['geo_data']]
generate_features(post_points)
post_points

In [ ]:
assert len(post_points.iloc[:,21:].columns) == (1 + len(coords_by_group)) * 4
feature_columns = post_points.iloc[:,21:].columns

In [556]:
post_points.to_csv(f'{point_type}.csv')